<a target="_blank" rel="noopener noreferrer" href="https://colab.research.google.com/github/shawnrhoads/gu-psyc-347/blob/master/docs/module-02-01_Nonlinear-Modeling.ipynb">![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)</a>

# Nonlinear Modeling

This tutorial was inspired by and adapted from the [Neuromatch Academy tutorials](https://github.com/NeuromatchAcademy/course-content) [[CC BY 4.0](https://creativecommons.org/licenses/by/4.0/)], using a nonlinear hyperbolic model to assess social discounting.

## Goals of this tutorial
1. [Specifying a nonlinear model](#what-is-a-nonlinear-model)
2. [Fitting data to a nonlinear model](#a-case-for-nonlinear-modeling-social-discounting)
3. [Comparing models](#model-comparison)
4. [Working with actual data](#fitting-actual-data-to-models)

In [ ]:
from scipy.optimize import minimize
from scipy import stats
import numpy as np, pandas as pd
import requests
import matplotlib.pyplot as plt
%matplotlib inline

## What is a nonlinear model?
Recall the general linear model, in which the multivariate relationship between a dependent variable ($y$) can be expressed as a linear combination of independent variables ($x_d$) that are multiplied by a weighted parameter or slope ($\beta_d$), plus some noise ($\epsilon$):

$$
y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + ... +\beta_d x_d + \epsilon
$$

where $\beta_0$ is the intercept and $d$ is the number of features (it is also the dimensionality of our input).

Nonlinear modeling simply implies that the relationship between $y$ and $x_d$ is more than a linear combination. Some common examples of nonlinear models include

**Sigmoid function**: 

$y=\frac{1}{1 + exp(\beta_0 + \beta_1x_1)}$

**Exponential function**: 

$y = \beta_0*exp(-\beta_1x_1)$

**Hyperbolic function**: 

$y=\frac{\beta_0}{1 + \beta_1x_1}$

In [ ]:
# Let's plot some of these examples:
np.random.seed(2021)

b0 = 1
b1 = .04
b2 = 1.5
b3 = .0125
b4 = 2.67

x1 = np.random.normal(10, 20,
                      size=(100,1))

noise = np.random.randn(100).reshape((100,1))

y = {'Linear': (b0 - b1*x1).reshape((100,1)),
     'Sigmoid': ( ( 1 / (1 + np.exp(b2 + b4*x1)) )).reshape((100,1)),
     'Exponential': (80*np.exp(-b1*x1)).reshape((100,1)),
     'Hyperbolic': ((80/(1 + b3*x1))).reshape((100,1))}

In [ ]:
fig, axes = plt.subplots(ncols=4, figsize=(18, 3))

for (key, values), ax in zip(y.items(), axes):
    
    # True data
    ax.scatter(x1, values)  # our data scatter plot

    ax.set(title= fr'{key}')
    
    ax.set_xticklabels('')
    ax.set_yticklabels('')
    ax.set_xlabel('x')
    ax.set_ylabel('y', rotation=0)

plt.show()

<hr>

## A case for nonlinear modeling: Social Discounting

Now that we have a better understanding about some nonlinear models, let's see how we can apply them to understand people's prosocial preferences towards close others versus distant strangers. We will fit different models to understand the phenomenon known as **social discounting**, which describes how the amount of resources that individuals are willing to sacrifice for others ($v$) decreases as a hyperbolic function of social distance ($N$) [(Jones & Rachlin, 2006)](https://journals.sagepub.com/doi/10.1111/j.1467-9280.2006.01699.x).

First, let's simulate some data to get a better intuition about the parameters in the models (the intercept $v_0$ and slope $k$):

$$v=\frac{v_0}{1 + k*N}$$

In [ ]:
np.random.seed(2021)

v0 = np.random.normal(85, 2,
                      size=(100,))

k = np.random.normal(.03, .005,
                      size=(100,))

N = np.array([1,2,5,10,20,50,100])

v = []
for i in range(100):
    v.append((v0[i] / (1 + k[i]*N)) + noise[i]) 
    plt.plot(N, v[i], alpha=.5)

plt.title('Social Discounting')
plt.ylabel('Amount Willing to Forgo (v)')
plt.xlabel('Social Distance (N)')
plt.show()

### Model Parameters

We have two "free" parameters in our hyperbolic model: the intercept ($v0$) and slope ($k$). The intercept ($v0$) represents the value of "amount willing to forgo" ($v$) when social distance ($N$) equals $0$. In other words, this is how much an individual values outcomes for him/herself (the undiscounted value of the outcomes)---a larger $v0$ would indicate that individuals value outcomes for themselves more (note that this parameter is somewhat difficult to interpret, so it is a common practice to add $+ 1$ to social distance so that the intercept represents the amount willing to forgo for $N=1$). The slope ($k$) measures the degree of discounting---a smaller $k$ describes more prosocial (or less selfish) choices for distant others while a larger $k$ describes more selfish (or less prosocial) choices for distant others.

To start off, let's create three functions in Python that compute $v$ (amount willing to forgo) as a function of $N$ (social distance). 

In both the exponential and hyperbolic case, we want to estimate the intercepts ($v_0$) and discounting rates ($k$) for each participant. We can compute these by minimizing the mean squared error (just like last time).

In [ ]:
def mse_linear(params, N, v):
    
    v0, k = params
    v_fit = v0 + k*N
    
    mse = np.mean((v - v_fit)**2)
    
    return mse   

In [ ]:
def mse_exponential(params, N, v):
    
    v0, k = params
    v_fit = v0*np.exp(-k*N)
    
    mse = np.mean((v - v_fit)**2)
    
    return mse    

In [ ]:
def mse_hyperbolic(params, N, v):
    
    v0, k = params
    v_fit = (v0 / (1 + k*N))
    
    mse = np.mean((v - v_fit)**2)
    
    return mse

In [ ]:
# initialize dictionary to store results
results = {"lin": [],
           "exp": [],
           "hyp": []}

# minimize MSE for linear function using scipy.optimize.minimize
results["lin"] = minimize(mse_linear, # objective function
                          (85, -.3), # estimated starting points
                          args=(N, v), # arguments
                          bounds=((50,100),(-1,1)),
                          tol=1e-3)

# minimize MSE for hyperbolic function using scipy.optimize.minimize
results["exp"] = minimize(mse_exponential, # objective function
                          (92, .6), # estimated starting points
                          args=(N, v), # arguments
                          bounds=((50,100),(0,1)),
                          tol=1e-3)

# minimize MSE for hyperbolic function using scipy.optimize.minimize
results["hyp"] = minimize(mse_hyperbolic, # objective function
                          (70, .05), # estimated starting points
                          args=(N, v), # arguments
                          bounds=((50,100),(0,1)),
                          tol=1e-3)

print(f'Linear: v0 = {results["lin"].x[0]:.2f}, k = {results["lin"].x[1]:.3f}, MSE = {results["lin"].fun:.2f}')
print(f'Exponential: v0 = {results["exp"].x[0]:.2f}, k = {results["exp"].x[1]:.3f}, MSE = {results["exp"].fun:.2f}')
print(f'Hyperbolic: v0 = {results["hyp"].x[0]:.2f}, k = {results["hyp"].x[1]:.3f}, MSE = {results["hyp"].fun:.2f}')

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(18, 4))
for (model, fits), ax in zip(results.items(), axes):

    # True data
    v_mean = np.mean(v, axis=0)
    ax.scatter(N, v_mean, 
               alpha = .75,
               label = 'Observed (mean)')  # our data scatter plot
    
    v0, k = fits.x
    mse_val = fits.fun

    # Compute and plot predictions
    if model == "lin":
        v_hat = v0 + k * N
    elif model == "exp":
        v_hat = v0*np.exp(-k*N)
    elif model == "hyp":
        v_hat = v0 / (1 + k*N)
        
    ax.plot(N, v_hat, color='r', label='Fit (mean)')  # our estimated model
    
    # plot residuals
    vmin = np.minimum(v_mean, v_hat)
    vmax = np.maximum(v_mean, v_hat)

    ax.vlines(N, vmin, vmax, 'g', alpha=0.5, label='Residuals')

    ax.set(
      title= fr'$k$= {k:.3f}, MSE = {mse_val:.2f}',
      xlabel='N',
      ylabel='v')
    
axes[0].legend()
axes[1].legend()
axes[2].legend()
plt.show()

Note that all of these model fits assume a **fixed** intercept ($v_o$) and **fixed** slope ($k$) across participants---in other words, these are the **average values** across the entire sample.

In reality, we know that intercepts and slopes vary across participants, and can plot the differences below.

In [ ]:
# plot all individual subjects
for v_subj in v:
    plt.plot(N, v_subj, alpha=.5)

# plot mean model fit
v_hat = results["hyp"].x[0] / (1 + results["hyp"].x[1]*N)        
plt.plot(N, v_hat, 
         color='black', 
         linewidth=3, 
         label='Fit (mean)')  # our estimated model

plt.title('Social Discounting')
plt.ylabel('Amount Willing to Forgo (v)')
plt.xlabel('Social Distance (N)')
plt.show()

In [ ]:
# See small differences between fitted and observed values
print(f'{v_hat} -- Fits')
print(f'{v_mean} -- Observed')

Thus, it might be better to make inferences with our data by allowing the intercepts and slopes to vary across participants. Fitting separate models for each participant is one way (of many) to accomplish this.

In [ ]:
# initialize a DataFrame, with columns corresponding to params ['v0', k] and rows corresponding to subjects
res_df = pd.DataFrame(columns=['v0', 'k'])

for subj_id, subj_v in enumerate(v):
    
    # minimize MSE for hyperbolic function using scipy.optimize.minimize
    fit = minimize(mse_hyperbolic, # objective function
                   (85, .05), # estimated starting points
                   args=(N, subj_v), # arguments
                   bounds=((50,100),(0,1)),
                   tol=1e-3)
        
    res_df.loc[subj_id, 'v0'] = fit.x[0]
    res_df.loc[subj_id, 'k'] = fit.x[1]
    res_df.loc[subj_id, 'MSE'] = fit.fun
    
    print(f'subject {subj_id}: v0 = {fit.x[0]:.2f}, k = {fit.x[1]:.3f}, MSE = {fit.fun:.2f}')

We can plot the distributions of $v_0$ and $k$ across participants

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(15,5))

axes[0].hist(res_df['v0'], bins=10)
axes[0].set(ylabel="Number of Subjects", xlabel="v0")

axes[1].hist(res_df['k'], bins=10)
axes[1].set(ylabel="Number of Subjects", xlabel="k")

plt.show()

<hr>

## Model comparison

Above, we can see that the hyperbolic model fits the data best, but typically the best fitting model isn't so obvious. Thus, we can use methods such as $R^2$ or the [Bayesian Information Criterion](https://en.wikipedia.org/wiki/Bayesian_information_criterion) ($BIC$) to compare model fits. The model with the lowest $BIC$ value is the best fitting model in a finite set of models.

The $BIC$ penalizes free parameters (see constant term: $k*ln(n)$):

$$ BIC = -2*ln(MSE) + p*ln(n)$$

Here, $n$ is the total number of observations in your sample (e.g., sample size), $p$ the number of parameters estimated by the model (we are estimating two parameters in our model: $v0$ and $k$), and $MSE$ is the mean sqauared error of the model. Remember that a lower $BIC$ value is better; Adding the term $p*ln(n)$ penalizes the model fit by the number of free parameters.

In [ ]:
def calculate_bic(sample_size, mse, num_params):
    bic = -2 * np.log(mse) + num_params * np.log(sample_size)
    return bic

In [ ]:
# calculate the Bayesian Information Criterion (bic)
for key, output in results.items():
    bic = calculate_bic(len(v_mean), output.fun, 2)
    print(f'{key} (BIC): {bic:.3f}')

We can now show that the hyperbolic model is the model with the lowest $BIC$ value (out of the models tested above).

<hr>

## Fitting actual data to models

Now that we've seen two examples of simulating data and model parameter recovery. Let's try to fit actual data to these models.

We will use a subset of the data from [Vekaria et al. (2017)](https://www.nature.com/articles/s41562-017-0100).

In [ ]:
# First let's load in the data

# here, we are just going to download data from the web (no need to edit these lines, but try to figure out what they are doing)
url = 'https://raw.githubusercontent.com/shawnrhoads/gu-psyc-347/master/docs/static/data/Vekaria-et-al-2017_data.csv'
df = pd.read_csv(url, index_col='subject')
print(df.head())

We can see that our data are formatted with participants as rows and amounts willing to forgo at each social distance as columns. To use our functions above, we will need to make sure our $v$ data have shape `(n_subjects, 7)`. 

Let's convert the pd.DataFrame to a np.array:

In [ ]:
vekaria_data = df.values

print(vekaria_data.shape)

First, let's fit all of the data together, with fixed intercepts and slopes, for both the hyperbolic model and the exponential model.

In [ ]:
fit1 = minimize(mse_hyperbolic, # objective function
               (85, .05), # estimated starting points
               args=(N, df.iloc[:,0:7].values), # arguments
               bounds=((0,80),(0,1)),
               tol=1e-3)

# minimize MSE for exponential function using scipy.optimize.minimize
fit2 = minimize(mse_exponential, # objective function
               (85, .05), # estimated starting points
               args=(N, df.iloc[:,0:7].values), # arguments
               bounds=((0,80),(0,1)),
               tol=1e-3)

In [ ]:
# fig, axes = plt.subplots()
fig = plt.figure()    

plt.scatter(N, np.mean(df.iloc[:,0:7].values, axis=0), label='Observed (mean)')
plt.plot(N, fit1.x[0] / (1 + fit1.x[1]*N), label='Hyperbolic')
plt.plot(N, fit2.x[0] * np.exp(-fit1.x[1]*N), label='Exponential')
plt.legend()

plt.show()

Based on this plot, we can clearly see how much better the hyperbolic model is at explaining the variance in the data. We can confirm this again using the $BIC$.

In [ ]:
# calculate the Bayesian Information Criterion (bic)
for label, output in zip(['hyperbolic', 'exponential'], [fit1, fit2]):
    bic = calculate_bic(len(vekaria_data), output.fun, 2)
    print(f'{label} (BIC): {bic:.3f}')

In [ ]:
# initialize a DataFrame, with columns corresponding to params ['v0', k] and rows corresponding to subjects
res_vekaria = pd.DataFrame(columns=['v0', 'k'])

for subj_id, subj_v in zip(df.index, vekaria_data):
    
    # minimize MSE for hyperbolic function using scipy.optimize.minimize
    fit = minimize(mse_hyperbolic, # objective function
                   (85, .05), # estimated starting points
                   args=(N, subj_v), # arguments
                   bounds=((0,80),(0,1)),
                   tol=1e-3)
        
    res_vekaria.loc[subj_id, 'v0'] = fit.x[0]
    res_vekaria.loc[subj_id, 'k'] = fit.x[1]
    res_vekaria.loc[subj_id, 'MSE'] = fit.fun
    
    print(f'subject {subj_id}: v0 = {fit.x[0]:.2f}, k = {fit.x[1]:.3f}, MSE = {fit.fun:.2f}')

We can see that some participants did not do very well with model fitting. For most, this is because their "amounts willing to forgo" do not vary across social distances.

To account for this, let's check which subjects these are.

In [ ]:
for subj_id, subj_v in zip(df.index, vekaria_data):
    if all(x==subj_v[0] for x in subj_v):
        print(f'no variation for subject #{subj_id}, {subj_v}')

Three participants sacrificed all of their resources for all social others. Let's assign `k=0` and `v0=85` to these participants since there is no variation in their preferences. This is eqivalent to a straight horizontal line (no discounting) at $v=85$.

In [ ]:
# initialize a DataFrame, with columns corresponding to params ['v0', k] and rows corresponding to subjects
hyp_vekaria = pd.DataFrame(columns=['v0', 'k'])

for subj_id, subj_v in zip(df.index, vekaria_data):
    
    if all(x==subj_v[0] for x in subj_v):
        if np.sum(subj_v)>=595:
            
            hyp_vekaria.loc[subj_id, 'v0'] = 80 # 
            hyp_vekaria.loc[subj_id, 'k'] = 0
            hyp_vekaria.loc[subj_id, 'MSE'] = np.nan
            
            print(f'assigning k=0 for subject #{subj_id}, {subj_v}')
    else:
    
        # minimize MSE for hyperbolic function using scipy.optimize.minimize
        fit = minimize(mse_hyperbolic, # objective function
                       (85, .05), # estimated starting points
                       args=(N, subj_v), # arguments
                       bounds=((0,80),(0,1)),
                       tol=1e-3)

        hyp_vekaria.loc[subj_id, 'v0'] = fit.x[0]
        hyp_vekaria.loc[subj_id, 'k'] = fit.x[1]
        hyp_vekaria.loc[subj_id, 'MSE'] = fit.fun

res_vekaria = pd.concat([df, hyp_vekaria], axis=1)

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(15,5), sharey=True)

axes[0].hist(res_vekaria['v0'], bins=20)
axes[0].set(ylabel="Number of Subjects", xlabel="v0")

axes[1].hist(res_vekaria['k'], bins=20)
axes[1].set(ylabel="Number of Subjects", xlabel="k")

plt.show()

Yay! Now, we can use these data for subsequent analyses! Note the little variation in $v_0$. Also note that $k$ is not parametric (e.g., not normally distributed), so we would need to conduct subsequent analyses using non-parametric approaches.